<a href="https://colab.research.google.com/github/Pandaklez/NLP/blob/master/classify_realfake_russian_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install razdel

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
from string import punctuation
from collections import Counter, defaultdict
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from nltk.tokenize import sent_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [2]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/lenta.txt.zip -O lenta.txt.zip
!unzip lenta.txt.zip

--2021-08-06 12:02:38--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/lenta.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5723675 (5.5M) [application/zip]
Saving to: ‘lenta.txt.zip’

lenta.txt.zip       100%[===================>]   5.46M  28.3MB/s    in 0.2s    

2021-08-06 12:02:38 (28.3 MB/s) - ‘lenta.txt.zip’ saved [5723675/5723675]

Archive:  lenta.txt.zip
replace lenta.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: lenta.txt               
replace __MACOSX/._lenta.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._lenta.txt    


In [24]:
#!wget https://ruscorpora.ru/new/ngrams/3grams-3.zip -O 3grams-3.zip
#!unzip 3grams-3.zip

--2021-08-06 11:04:24--  https://ruscorpora.ru/new/ngrams/3grams-3.zip
Resolving ruscorpora.ru (ruscorpora.ru)... 82.148.12.99
Connecting to ruscorpora.ru (ruscorpora.ru)|82.148.12.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32452212 (31M) [application/zip]
Saving to: ‘3grams-3.zip’

3grams-3.zip        100%[===================>]  30.95M  7.26MB/s    in 4.3s    

2021-08-06 11:04:29 (7.26 MB/s) - ‘3grams-3.zip’ saved [32452212/32452212]

Archive:  3grams-3.zip
  inflating: 3grams-3.txt            


In [14]:
!ls -lsha

total 215M
4.0K drwxr-xr-x 1 root root 4.0K Aug  6 11:15 .
4.0K drwxr-xr-x 1 root root 4.0K Aug  6 10:37 ..
158M -rw-r--r-- 1 root root 158M Nov  1  2011 3grams-3.txt
 31M -rw-r--r-- 1 root root  31M Aug  3 17:55 3grams-3.zip
4.0K drwxr-xr-x 4 root root 4.0K Jul 16 13:19 .config
 21M -rw-r--r-- 1 root root  21M Nov 15  2020 lenta.txt
5.5M -rw-r--r-- 1 root root 5.5M Aug  6 11:15 lenta.txt.zip
4.0K drwxr-xr-x 2 root root 4.0K Aug  6 11:15 __MACOSX
4.0K drwxr-xr-x 1 root root 4.0K Jul 16 13:20 sample_data


In [3]:
with open('lenta.txt', 'r', encoding='utf-8') as f:
    lenta = f.read()

#lenta = lenta[:50000]

In [4]:
len(lenta)
lenta[:500]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомоб'

In [37]:
# количество токенов в корпусе
len(normalize(lenta))

1505789

Мы хотим сделать корпус на 10 миллионов слов..

In [42]:
with open('mal_hik_so_spagoi.txt', 'r', encoding='utf-8') as f:
    spaga = f.read()

with open('nabokov.txt', 'r', encoding='utf-8') as f:
    nabokov = f.read()

with open('wiki_data.txt', 'r', encoding='utf-8') as f:
  wiki = f.read()

In [43]:
all_texts = lenta + '\n' + spaga + '\n' + nabokov + '\n' + wiki

In [44]:
len(normalize(all_texts))

7087345

Вот мы прочитали данные (новостные тексты ленты), теперь мы можем сделать фейковые предложения на русском языке, например, на триграммах

In [45]:
def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

def create_ngrams(sentences):
    unigrams, bigrams, trigrams = Counter(), Counter(), Counter()
    for sentence in sentences:
        unigrams.update(sentence)
        bigrams.update(ngrammer(sentence, n=2))
        trigrams.update(ngrammer(sentence, n=3))
    return unigrams, bigrams, trigrams

In [46]:
sentences_news = [ ['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(all_texts)]

unigrams_news, bigrams_news, trigrams_news = create_ngrams(sentences_news)

In [47]:
len(sentences_news)

305528

In [8]:
def create_matrix_old(unigrams, bigrams, trigrams):
    matrix_trigrams = np.zeros((len(bigrams), len(unigrams)))
    matrix_bigrams = np.zeros((len(unigrams), len(unigrams)))
    id2bigram = list(bigrams)
    bigram2id  =  {bigram:i for i, bigram in enumerate(id2bigram)}
    id2word = list(unigrams)
    word2id = {word:i for i, word in enumerate(id2word)}
    for ngram in trigrams:
        ngram_ = ngram.split(' ')
        bigram, unigram = ' '.join(ngram_[:-1]), ngram_[-1]
        matrix_trigrams[bigram2id[bigram]][word2id[unigram]] =  (trigrams[ngram] / bigrams[bigram])
    for ngram in bigrams:
        word1, word2 = ngram.split(' ')
        matrix_bigrams[word2id[word1]][word2id[word2]] =  (bigrams[ngram] / unigrams[word1])
    return matrix_trigrams, matrix_bigrams, id2bigram, bigram2id, id2word, word2id

In [9]:
list(trigrams_news)[:5]

['<start> <start> бои',
 '<start> бои у',
 'бои у сопоцкина',
 'у сопоцкина и',
 'сопоцкина и друскеник']

In [28]:
# bigram2id_news
{bigram:i for i, bigram in enumerate(list(bigrams_news)[:10])}

{'<start> <start>': 0,
 '<start> бои': 1,
 'бои у': 2,
 'германцев <end>': 9,
 'друскеник закончились': 6,
 'закончились отступлением': 7,
 'и друскеник': 5,
 'отступлением германцев': 8,
 'сопоцкина и': 4,
 'у сопоцкина': 3}

In [31]:
trigrams_news['30 раненых в']/bigrams_news['30 раненых']

0.5

In [48]:
def create_matrix(unigrams, bigrams, trigrams):
    # matrix_trigrams = np.zeros((len(bigrams), len(unigrams)))
    # matrix_bigrams = np.zeros((len(unigrams), len(unigrams)))
    dict_trigrams = defaultdict(int)

    id2bigram = list(bigrams)
    bigram2id  =  {bigram:i for i, bigram in enumerate(id2bigram)}
    id2word = list(unigrams)
    word2id = {word:i for i, word in enumerate(id2word)}
    for ngram in trigrams:
        ngram_ = ngram.split(' ')
        bigram, unigram = ' '.join(ngram_[:-1]), ngram_[-1]
        # bigram бои у || unigram сопоцкина
        # matrix_trigrams[bigram2id[bigram]][word2id[unigram]] =  (trigrams[ngram] / bigrams[bigram])
        dict_trigrams[bigram + '_' + unigram] = (trigrams[ngram] / bigrams[bigram])
    #for ngram in bigrams:
    #    word1, word2 = ngram.split(' ')
    #    matrix_bigrams[word2id[word1]][word2id[word2]] =  (bigrams[ngram] / unigrams[word1])
    return dict_trigrams, id2bigram, bigram2id, id2word, word2id  # matrix_bigrams

In [49]:
dict_trigrams_news, id2bigram_news, bigram2id_news, id2word_news, word2id_news = create_matrix(unigrams_news, bigrams_news, trigrams_news)

In [ ]:
dict_trigrams_news

In [16]:
'<start> <start>' + '_' + 'агентство' in dict_trigrams_news.keys()

True

In [22]:
# list of unigrams
id2word_news[3]

'сопоцкина'

In [ ]:
id2bigram_news

In [ ]:
def trigrams_generator_old(matrix, id2word, ngram2id, id2ngram, n=100, start='<start> <start>'):
    text = []
    current_idx = ngram2id[start]
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        if id2word[chosen] == '<end>':
            token = start
        else:
            token = id2ngram[current_idx].split()[-1] + " " + id2word[chosen]
        current_idx = ngram2id[token]
    return ' '.join(text)

In [55]:
def trigrams_generator(dict_grams, id2word, ngram2id, id2ngram, n=30, start='<start> <start>'):
    text = []
    current_idx = ngram2id[start]
    current_token = start

    for i in range(n):
        # chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        probs = [dict_grams[current_token + '_' + word] if current_token + '_' + word in dict_grams.keys() else 0.0 for word in id2word]
        chosen = np.random.choice(len(id2word), p=probs)
        text.append(id2word[chosen])
        if id2word[chosen] == '<end>':
            current_token = start
        else:
            # get current bigram, get last word of current bigram, glue last word of bigram + chosen token
            current_token = id2ngram[current_idx].split()[-1] + " " + id2word[chosen]
        current_idx = ngram2id[current_token]
    return ' '.join(text)

In [57]:
from pprint import pprint

print(trigrams_generator(dict_grams=dict_trigrams_news, id2word=id2word_news, ngram2id=bigram2id_news, id2ngram=id2bigram_news).replace('<end>', '\n'))

философское антиковедение и классическая музыка пение плавание чтение биографической и исторической лингвистики леману принадлежит заслуга основания двух кафедр в техасском университете 
 перси аннабет 
 также лемоннье в чём не
